<a href="https://colab.research.google.com/github/JobinJose9660/sales-predictions/blob/main/machine_learing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector,make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

In [2]:
filename  = '/content/sales_predictions(5).csv'
sales  = pd.read_csv(filename)

In [3]:
sales['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [4]:
sales.loc[sales['Item_Fat_Content'] == 'LF', 'Item_Fat_Content'] = 'Low Fat'

sales.loc[sales['Item_Fat_Content'] == 'reg', 'Item_Fat_Content'] = 'Regular'

sales.loc[sales['Item_Fat_Content'] == 'low fat', 'Item_Fat_Content'] = 'Low Fat'

sales.loc[sales['Item_Fat_Content'] == 'LF', 'Item_Fat_Content'] = 'Low Fat'

sales.loc[sales['Item_Fat_Content'] == 'reg', 'Item_Fat_Content'] = 'Regular'

sales.loc[sales['Item_Fat_Content'] == 'low fat', 'Item_Fat_Content'] = 'Low Fat'

In [5]:
X= sales.drop(columns='Item_Outlet_Sales')
y = sales['Item_Outlet_Sales']
X_train, X_test ,y_train,y_test = train_test_split(X,y,random_state = 42)

In [6]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [7]:
numeric_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

numeric_imputer.fit(X_train[['Item_Weight']])
X_train['Item_Weight'] = numeric_imputer.transform(X_train[['Item_Weight']])
X_test['Item_Weight'] = numeric_imputer.transform(X_test[['Item_Weight']])


categorical_imputer.fit(X_train[['Outlet_Size']])
X_train['Outlet_Size'] = categorical_imputer.transform(X_train[['Outlet_Size']])
X_test['Outlet_Size'] = categorical_imputer.transform(X_test[['Outlet_Size']])

In [8]:
X_train.isnull().any()

Item_Identifier              False
Item_Weight                  False
Item_Fat_Content             False
Item_Visibility              False
Item_Type                    False
Item_MRP                     False
Outlet_Identifier            False
Outlet_Establishment_Year    False
Outlet_Size                  False
Outlet_Location_Type         False
Outlet_Type                  False
dtype: bool

In [9]:
sizes = {'Small':0,'Medium':1,'High':2}
X_train['Outlet_Size'] = X_train['Outlet_Size'].replace(sizes)

In [10]:
X_train['Outlet_Size'].value_counts()

1    3915
0    1788
2     689
Name: Outlet_Size, dtype: int64

In [11]:
X_train['Outlet_Size'] = X_train['Outlet_Size'].astype('object')
X_test['Outlet_Size'] =X_test['Outlet_Size'].astype('object')

In [12]:
encoder = OneHotEncoder(sparse=False,handle_unknown='ignore')
scalaer = StandardScaler()

In [13]:
ohe_tuple = (encoder,cat_selector)
scalar_tuple =(scalaer,num_selector)
column_transform = make_column_transformer(ohe_tuple,scalar_tuple,remainder = 'passthrough')

In [14]:
lin_reg = LinearRegression()
pipe = make_pipeline(column_transform,lin_reg)
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb38ae8a090>),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fb38ae8a050>)])),
                ('linearregression', LinearRegression())])

In [15]:
from sklearn.metrics import r2_score
print(f'Train R2: {r2_score(y_train, pipe.predict(X_train))}')
print(f'Test R2: {r2_score(y_test, pipe.predict(X_test))}')

Train R2: 0.6702241974534979
Test R2: -1.1548236158716282e+25
